# 1948. Delete Duplicate Folders in System

**Hard**

Due to a bug, there are many duplicate folders in a file system. You are given a 2D array paths, where paths[i] is an array representing an absolute path to the ith folder in the file system.

- For example, ["one", "two", "three"] represents the path "/one/two/three".

Two folders (not necessarily on the same level) are identical if they contain the same non-empty set of identical subfolders and underlying subfolder structure. The folders do not need to be at the root level to be identical. If two or more folders are identical, then mark the folders as well as all their subfolders.

- For example, folders "/a" and "/b" in the file structure below are identical. They (as well as their subfolders) should all be marked:
- /a
- /a/x
- /a/x/y
- /a/z
- /b
- /b/x
- /b/x/y
- /b/z

- However, if the file structure also included the path "/b/w", then the folders "/a" and "/b" would not be identical. Note that "/a/x" and "/b/x" would still be considered identical even with the added folder.

Once all the identical folders and their subfolders have been marked, the file system will delete all of them. The file system only runs the deletion once, so any folders that become identical after the initial deletion are not deleted.

Return the 2D array ans containing the paths of the remaining folders after deleting all the marked folders. The paths may be returned in any order.

# Example 1:

```python

Input: paths = [["a"],["c"],["d"],["a","b"],["c","b"],["d","a"]]
Output: [["d"],["d","a"]]
```

**Explanation:** The file structure is as shown.
Folders "/a" and "/c" (and their subfolders) are marked for deletion because they both contain an empty
folder named "b".

# Example 2:

```python

Input: paths = [["a"],["c"],["a","b"],["c","b"],["a","b","x"],["a","b","x","y"],["w"],["w","y"]]
Output: [["c"],["c","b"],["a"],["a","b"]]
```

**Explanation:** The file structure is as shown.
Folders "/a/b/x" and "/w" (and their subfolders) are marked for deletion because they both contain an empty folder named "y".
Note that folders "/a" and "/c" are identical after the deletion, but they are not deleted because they were not marked beforehand.

# Example 3:

```python

Input: paths = [["a","b"],["c","d"],["c"],["a"]]
Output: [["c"],["c","d"],["a"],["a","b"]]
```

Explanation: All folders are unique in the file system.
Note that the returned array can be in a different order as the order does not matter.

**Constraints:**

- 1 <= paths.length <= 2 \_ 104
- 1 <= paths[i].length <= 500
- 1 <= paths[i][j].length <= 10
- 1 <= sum(paths[i][j].length) <= 2 \_ 105
- path[i][j] consists of lowercase English letters.
- No two paths lead to the same folder.
- For any folder not at the root level, its parent folder will also be in the input.


In [ ]:
from collections import defaultdict

class TrieNode:
    def __init__(self, folder_name):
        self.folder_name = folder_name
        self.children = dict()
        self.marked_for_removal = False

class Solution:
    def insert_into_trie(self, root, folder_path):
        node = root
        for folder in folder_path:
            if folder not in node.children:
                node.children[folder] = TrieNode(folder)
            node = node.children[folder]

    def encode_subtree(self, node, subtree_map):
        subtree_encoding = ""
        for child_name, child_node in sorted(node.children.items()):
            subtree_encoding += self.encode_subtree(child_node, subtree_map)

        if subtree_encoding:
            if subtree_encoding in subtree_map:
                subtree_map[subtree_encoding].marked_for_removal = True
                node.marked_for_removal = True
            else:
                subtree_map[subtree_encoding] = node

        return "[" + node.folder_name + subtree_encoding + "]"

    def collect_valid_paths(self, node, current_path, result_paths):
        if node.marked_for_removal:
            return

        current_path.append(node.folder_name)
        result_paths.append(current_path[:])  # deep copy for isolation
        for child_name, child_node in node.children.items():
            self.collect_valid_paths(child_node, current_path, result_paths)
        current_path.pop()

    def deleteDuplicateFolder(self, folder_paths):
        root = TrieNode("/")
        for path in folder_paths:
            self.insert_into_trie(root, path)

        subtree_map = dict()
        self.encode_subtree(root, subtree_map)

        result_paths = []
        current_path = []
        for child_name, child_node in root.children.items():
            self.collect_valid_paths(child_node, current_path, result_paths)

        return result_paths